In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Data Preprocessing

In [ ]:
#Inisiasi Data dari .csv
stock1 = pd.DataFrame(pd.read_csv("E:\AKTUARIA SMT 7\SIMULASI AKTUARIA\DCII.JK.csv", sep=','))
stock2 = pd.DataFrame(pd.read_csv("E:\AKTUARIA SMT 7\SIMULASI AKTUARIA\ICBP.JK.csv", sep=','))
stock3 = pd.DataFrame(pd.read_csv("E:\AKTUARIA SMT 7\SIMULASI AKTUARIA\BBCA.JK.csv", sep=','))

#Mengambil data harga saham saat penutupan
close_stock1 = stock1['Close']
close_stock2 = stock2['Close']
close_stock3 = stock3['Close']

#Menggabungkan data harga saham penutupan
data = pd.concat([close_stock1, close_stock2, close_stock3], axis=1)
data.columns = ['DCII', 'ICBP', 'BBCA']

#Mendapatkan tingkat keuntungan (return) berdasarkan harga saham penutupan
return_data = data.pct_change()
return_data = return_data.dropna() #Menghilangkan data dengan value NA
print(return_data)

# Menentukan Proporsi Saham Terbaik

In [ ]:
#Inisiasi variabel pendukung
jumlah_data_simulasi = 1000
results = np.zeros((3, jumlah_data_simulasi))
risks = np.zeros(jumlah_data_simulasi)
returns = np.zeros(jumlah_data_simulasi)
prop = np.zeros((jumlah_data_simulasi, 3))
dot = []
np.random.seed(1)
#Iterasi untuk generate nilai return gabungan sebanyak jumlah data simulasi yang ditentukan
for i in range(jumlah_data_simulasi):
    proporsi = np.random.random(3)
    proporsi /= np.sum(proporsi)
    prop[i, 0] = proporsi[0]
    prop[i, 1] = proporsi[1]
    prop[i, 2] = proporsi[2]

    for j in range(len(return_data)):
        dot.append(np.dot(proporsi, return_data.iloc[j]))
    
    portfolio_return = np.mean(dot)
    portfolio_risk = np.std(dot)
    
    returns[i] = portfolio_return
    risks[i] = portfolio_risk
    results[0, i] = proporsi[0]
    results[1, i] = proporsi[1]
    results[2, i] = proporsi[2]

utilities = returns - 0.5 * risks**2
max_utility_idx = np.argmax(utilities)
proporsi_optimal = results[:, max_utility_idx]
print('Proporsi optimal (Markowitz): ', proporsi_optimal)
print('Return pada Proporsi Optimal: ', round(returns[max_utility_idx]*100,2), '%')

# Visualisasi hasil simulasi nilai return gabungan
plt.scatter(risks, returns, c=utilities, marker='o', cmap='viridis')
plt.xlabel('Risiko')
plt.ylabel('Return')
plt.colorbar(label='Utilitas (Markowitz)')
plt.scatter(risks[max_utility_idx], returns[max_utility_idx], marker='x', color='red', s=200, label='Optimal (Markowitz)')
plt.legend()
plt.show()

In [ ]:
#Mendapatkan tabel return/risk hasil bangkitan
return_risk = pd.DataFrame({'Proporsi DCII' : prop[:,0],
                            'Proporsi ICBP' : prop[:,1],
                            'Proporsi BBCA' : prop[:,2],
                            'Return' : returns,
                            'Risiko' : risks})
return_risk.to_excel('Return dan Risiko Gabungan.xlsx', index = True)

# Monte Carlo Simulation

In [ ]:
#Menggunakan metode Empirical CDF agar distribusi data saham dapat digunakan pada generator angka acak dalam monte carlo 
def ecdf(data):
    sorted_data = np.sort(data)
    n = len(data)
    y = np.arange(1, n + 1) / n
    return sorted_data, y

return_gabungan = np.dot(return_data.values, proporsi_optimal)
x_gabungan, y_gabungan = ecdf(return_gabungan)

#Melakukan simulasi monte carlo terhadap data return gabungan saham dengan proporsi optimal
np.random.seed(1)
montecarlosim = []
for _ in range(len(return_data)):
    uniform_sample = np.random.rand()
    result = np.interp(uniform_sample, y_gabungan, x_gabungan)
    montecarlosim.append(result)
montecarlosim = np.array(montecarlosim)


# Value at Risk

In [ ]:
#Perhitungan Value at Risk awal pada data hasil simulasi monte carlo
alpha = ([0.01, 0.05, 0.10])
val_at_risk = pd.DataFrame()
val_at_risk['Value'] = (montecarlosim.mean() - (montecarlosim.std() * norm.ppf(alpha)))
print(val_at_risk)

# Perhitungan Variasi Nilai VaR

In [ ]:
#Melakukan simulasi variasi nilai VaR pada tingkat kepercayaan 99%, 95%, dan 90%
def var_variation(ValueatRisk, Iterasi):
    var_table = pd.DataFrame()
    np.random.seed(1)
    #Iterasi untuk mendapatkan variasi nilai VaR secara random
    for i in range(Iterasi):
        random_factors = np.random.rand(ValueatRisk.size) / 100
        variations = ValueatRisk.values.flatten() + random_factors
        var_table[i] = variations
    
    var_table = var_table.T
    return var_table

var_sim = var_variation(val_at_risk, 1000) #Menyimpan nilai VaR hasil simulasi
var_100 = var_sim.mean() * 1000000000 #Mendapatkan nilai VaR apabila investasi sebesar Rp100.000.000,- dilakukan pada portofolio 

In [ ]:
var_mean = pd.DataFrame(var_sim.mean())
label = pd.DataFrame(alpha)
z_score = pd.DataFrame(abs(norm.ppf(alpha)))
var_summary = pd.concat([label, z_score, val_at_risk, var_mean], axis=1)
var_summary.columns = ['Alpha', 'Z-Score', 'VaR', 'Mean VaR']
def kesimpulan():
    print('Profil Portofolio')
    print('Proporsi Saham Optimal: ', proporsi_optimal)
    print('Expected Return (Mean): ', (round((return_gabungan.mean()*100),3)), '%')
    print('Risiko (Std. Deviasi): ', (round(return_gabungan.std(),3)), '\n')
    print('Evaluasi Hasil Simulasi Monte Carlo')
    print('Expected Return (Mean): ', (round((montecarlosim.mean()*100),3)), '%')
    print('Risiko (Std. Deviasi): ', (round(montecarlosim.std(),3)), '\n')
    print('Evaluasi Perhitungan Value at Risk')
    print(var_summary)

kesimpulan()

#Mendapatkan tabel VaR
var_summary.to_excel('VaR Gabungan.xlsx', index = True)